# Observability

이 워크샵에서는 Agent Framework에서 OpenTelemetry를 사용하여 AI 애플리케이션을 관찰하고 모니터링하는 방법을 학습합니다.

## 목차
1. [Observability란 무엇인가?](#observability란-무엇인가)
2. [환경 설정](#환경-설정)
3. [기본 Observability 설정](#기본-observability-설정)
4. [ChatAgent Observability](#chatagent-observability)
5. [Azure AI Agent Observability](#azure-ai-agent-observability)
6. [Custom Span 생성](#custom-span-생성)
7. [Workflow Observability](#workflow-observability)
8. [Application Insights 연동](#application-insights-연동)

## 학습 목표
- OpenTelemetry와 Observability 개념 이해
- Agent Framework의 자동 계측(instrumentation) 활용
- Traces, Logs, Metrics 수집 및 분석
- Azure Application Insights 연동
- Aspire Dashboard를 통한 로컬 개발 모니터링

## 1. Observability란 무엇인가?

**Observability**는 시스템의 내부 상태를 외부 출력을 통해 이해할 수 있는 능력입니다.

### OpenTelemetry 3대 요소

1. **Traces (추적)** 🔍
   - 요청이 시스템을 통과하는 전체 경로
   - Span: 단일 작업 단위
   - Agent 실행, 함수 호출, AI 모델 호출 등
   - 분산 시스템의 성능 병목 지점 파악

2. **Logs (로그)** 📝
   - 특정 시점의 이벤트 기록
   - 디버깅 및 문제 해결
   - 컨텍스트 정보 포함
   - 로그 레벨: DEBUG, INFO, WARNING, ERROR

3. **Metrics (메트릭)** 📊
   - 시간 경과에 따른 수치 데이터
   - 토큰 사용량, 응답 시간, 에러율
   - 성능 모니터링 및 알림
   - 리소스 사용량 추적

### Agent Framework Observability

Agent Framework는 **OpenTelemetry Semantic Conventions for GenAI**를 따라 자동으로 telemetry를 생성합니다:

- ✅ **Agent/Model 호출**: 모든 AI 모델 호출 자동 추적
- ✅ **Tool 실행**: 함수 호출 및 실행 시간 기록
- ✅ **토큰 소비**: 입력/출력 토큰 수 자동 집계
- ✅ **에러 추적**: 예외 및 실패 자동 로깅
- ✅ **워크플로우**: End-to-end 가시성

### Telemetry 대상

- 🖥️ **Console**: 로컬 개발 및 디버깅
- 📊 **Aspire Dashboard**: Docker 기반 로컬 모니터링
- ☁️ **Azure Application Insights**: 프로덕션 모니터링
- 🔧 **기타 APM**: Prometheus, Jaeger, Grafana 등

## 2. 환경 설정

필요한 패키지를 설치하고 환경 변수를 설정합니다.

In [1]:
# 필요한 패키지 설치 (이미 포함되어 있음)
# agent-framework에 OpenTelemetry 패키지가 포함되어 있어 추가 설치 불필요
# !pip install agent-framework agent-framework-azure-ai python-dotenv

In [2]:
# 환경 변수 로드
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드 (override=True로 기존 값 덮어쓰기)
load_dotenv(override=True)

# Observability 필수 환경 변수
observability_vars = {
    "ENABLE_OTEL": "Observability 활성화 (true/false)",
    "ENABLE_SENSITIVE_DATA": "민감한 데이터 포함 (true/false, 개발용)",
    "OTLP_ENDPOINT": "OTLP 엔드포인트 (Aspire Dashboard 등)",
    "APPLICATIONINSIGHTS_CONNECTION_STRING": "Application Insights 연결 문자열",
}

print("=== Observability 환경 변수 ===")
for var, description in observability_vars.items():
    value = os.getenv(var)
    status = "✓" if value else "○"
    display_value = "(설정됨)" if value and "CONNECTION" in var else (value or "(미설정)")
    print(f"{status} {var}: {display_value}")
    print(f"   -> {description}")

# 일반 환경 변수
print("\n=== 일반 환경 변수 ===")
required_vars = [
    "OPENAI_API_KEY",
    "AZURE_AI_PROJECT_ENDPOINT",
]

for var in required_vars:
    status = "✓" if os.getenv(var) else "✗"
    print(f"{status} {var}")

=== Observability 환경 변수 ===
✓ ENABLE_OTEL: true
   -> Observability 활성화 (true/false)
✓ ENABLE_SENSITIVE_DATA: true
   -> 민감한 데이터 포함 (true/false, 개발용)
✓ OTLP_ENDPOINT: http://localhost:4317/
   -> OTLP 엔드포인트 (Aspire Dashboard 등)
○ APPLICATIONINSIGHTS_CONNECTION_STRING: (미설정)
   -> Application Insights 연결 문자열

=== 일반 환경 변수 ===
✗ OPENAI_API_KEY
✓ AZURE_AI_PROJECT_ENDPOINT


In [3]:
# 공통 import
import asyncio
import datetime
from random import randint
from typing import Annotated

from agent_framework import ChatAgent
from agent_framework.observability import get_tracer, setup_observability
from agent_framework.openai import OpenAIChatClient
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

from opentelemetry.trace import SpanKind
from opentelemetry.trace.span import format_trace_id
from pydantic import Field

print("✓ Import 완료")

✓ Import 완료


## 3. 기본 Observability 설정

`setup_observability()` 함수를 사용하여 간단하게 observability를 활성화할 수 있습니다.

### 3.1 도구 함수 정의

In [4]:
async def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """지정된 위치의 날씨를 반환합니다."""
    # 네트워크 호출 시뮬레이션
    await asyncio.sleep(randint(0, 10) / 10.0)
    conditions = ["맑음", "흐림", "비", "폭풍"]
    return f"{location}의 날씨는 {conditions[randint(0, 3)]}이며 최고 기온은 {randint(10, 30)}°C입니다."


def get_current_time() -> str:
    """현재 시간을 반환합니다."""
    return f"현재 시간은 {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}입니다."


print("✓ 도구 함수 정의 완료")

✓ 도구 함수 정의 완료


### 3.2 환경 변수 기반 설정

`setup_observability()`는 환경 변수를 기반으로 자동으로 exporter를 설정합니다:

- `OTLP_ENDPOINT`: OTLP 프로토콜 사용 (Aspire Dashboard, Jaeger 등)
- `APPLICATIONINSIGHTS_CONNECTION_STRING`: Azure Application Insights 사용
- 둘 다 없으면: Console 출력 (기본)

In [5]:
# Observability 설정
# 환경 변수 기반으로 자동 설정됨
setup_observability()

print("✓ Observability 설정 완료")
print("  - ENABLE_OTEL 또는 ENABLE_SENSITIVE_DATA가 true이면 활성화됩니다.")
print("  - OTLP_ENDPOINT 또는 APPLICATIONINSIGHTS_CONNECTION_STRING로 대상 지정")
print("  - 지정된 exporter가 없으면 Console에 출력됩니다.")

✓ Observability 설정 완료
  - ENABLE_OTEL 또는 ENABLE_SENSITIVE_DATA가 true이면 활성화됩니다.
  - OTLP_ENDPOINT 또는 APPLICATIONINSIGHTS_CONNECTION_STRING로 대상 지정
  - 지정된 exporter가 없으면 Console에 출력됩니다.


### 3.3 간단한 Observability 테스트

In [ ]:
async def test_basic_observability():
    """기본 observability 테스트"""
    print("=== 기본 Observability 테스트 ===")
    
    # Custom span 생성 - 전체 시나리오를 감싸는 최상위 span
    with get_tracer().start_as_current_span(
        "시나리오: 기본 Agent 채팅",
        kind=SpanKind.CLIENT
    ) as current_span:
        # Trace ID 출력 (이 ID로 전체 요청 추적 가능)
        trace_id = format_trace_id(current_span.get_span_context().trace_id)
        print(f"\n🔍 Trace ID: {trace_id}")
        print("   -> Application Insights나 Aspire Dashboard에서 이 ID로 검색 가능\n")
        
        # Agent 생성
        agent = ChatAgent(
            chat_client=OpenAIChatClient(),
            tools=[get_weather, get_current_time],
            name="WeatherAgent",
            instructions="당신은 날씨와 시간을 알려주는 도움이 되는 조수입니다.",
        )
        
        # 질문 리스트
        questions = [
            "서울의 날씨는 어때?",
            "지금 몇 시야?",
        ]
        
        for question in questions:
            print(f"[사용자] {question}")
            print(f"[{agent.display_name}] ", end="")
            
            # 스트리밍 응답 (각 청크가 자동으로 trace됨)
            async for update in agent.run_stream(question):
                if update.text:
                    print(update.text, end="")
            print("\n")
        
        print("\n✓ 테스트 완료")
        print("\n📊 수집된 Telemetry:")
        print("   - Traces: Agent 실행, Tool 호출, AI 모델 호출")
        print("   - Logs: 실행 로그, 에러 로그")
        print("   - Metrics: 토큰 사용량, 응답 시간")

# 실행
await test_basic_observability()

=== 기본 Observability 테스트 ===

🔍 Trace ID: 171896f9ae2fb96e3cc204c1ce96dd25
   -> Application Insights나 Aspire Dashboard에서 이 ID로 검색 가능



ServiceInitializationError: OpenAI API key is required. Set via 'api_key' parameter or 'OPENAI_API_KEY' environment variable.

[2025-11-04 09:06:28 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/opentelemetry/exporter/otlp/proto/grpc/exporter.py:424 - WARNING] Transient error StatusCode.UNAVAILABLE encountered while exporting traces to localhost:4317, retrying in 0.90s.
[2025-11-04 09:06:35 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/opentelemetry/exporter/otlp/proto/grpc/exporter.py:416 - ERROR] Failed to export traces to localhost:4317, error code: StatusCode.UNAVAILABLE
[2025-11-04 09:06:36 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/opentelemetry/exporter/otlp/proto/grpc/exporter.py:424 - WARNING] Transient error StatusCode.UNAVAILABLE encountered while exporting logs to localhost:4317, retrying in 3.29s.
[2025-11-04 09:06:39 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/opentelemetry/exporter/otlp/proto/grpc/exporter.py:416 - ERROR] Failed to export logs to localhost:4317, error code: StatusCode.UNAVAILA

## 4. ChatAgent Observability

ChatAgent를 사용할 때 자동으로 수집되는 telemetry를 살펴봅니다.

### 4.1 Thread 기반 대화

In [ ]:
async def test_chat_agent_with_thread():
    """Thread를 사용한 다중 턴 대화 observability"""
    print("=== ChatAgent with Thread Observability ===")
    
    with get_tracer().start_as_current_span(
        "시나리오: Multi-turn 대화",
        kind=SpanKind.CLIENT
    ) as current_span:
        trace_id = format_trace_id(current_span.get_span_context().trace_id)
        print(f"\n🔍 Trace ID: {trace_id}\n")
        
        # Agent 생성
        agent = ChatAgent(
            chat_client=OpenAIChatClient(),
            tools=get_weather,
            name="WeatherAgent",
            instructions="당신은 날씨 정보를 제공하는 전문가입니다. 사용자의 위치를 기억하세요.",
        )
        
        # Thread 생성 - 대화 컨텍스트 유지
        thread = agent.get_new_thread()
        
        # 다중 턴 대화
        questions = [
            "서울의 날씨는 어때?",
            "거기랑 부산이랑 어디가 더 좋아?",  # 이전 컨텍스트 활용
            "하늘은 왜 파래?",  # 도구가 필요 없는 질문
        ]
        
        for i, question in enumerate(questions, 1):
            print(f"\n--- 턴 {i} ---")
            print(f"[사용자] {question}")
            print(f"[{agent.display_name}] ", end="")
            
            # 같은 thread를 사용하여 컨텍스트 유지
            async for update in agent.run_stream(question, thread=thread):
                if update.text:
                    print(update.text, end="")
            print("")
        
        print("\n✓ Multi-turn 대화 완료")
        print("\n📊 각 턴마다 개별 span이 생성되어 전체 대화 흐름을 추적할 수 있습니다.")

# 실행
await test_chat_agent_with_thread()

## 5. Azure AI Agent Observability

Azure AI를 사용할 때는 `setup_azure_ai_observability()`를 사용하여 Application Insights와 자동으로 연동됩니다.

### 5.1 Azure AI Agent Client 설정

In [ ]:
async def test_azure_ai_observability():
    """Azure AI Agent의 자동 observability 설정"""
    print("=== Azure AI Agent Observability ===")
    
    # Azure AI Project가 필요합니다
    if not os.getenv("AZURE_AI_PROJECT_ENDPOINT"):
        print("⚠️  AZURE_AI_PROJECT_ENDPOINT가 설정되지 않았습니다.")
        print("   이 예제를 실행하려면 Azure AI Project가 필요합니다.")
        return
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential) as client,
    ):
        # Azure AI Project에 연결된 Application Insights로 자동 설정
        # 이 메서드는:
        # 1. Azure AI Project에서 Application Insights 연결 문자열을 가져옴
        # 2. Observability를 자동으로 설정
        # 3. 기존 설정을 덮어씀
        await client.setup_azure_ai_observability()
        
        print("✓ Azure AI Observability 설정 완료")
        print("  -> Application Insights로 자동 연결됨\n")
        
        with get_tracer().start_as_current_span(
            "시나리오: Azure AI Agent 채팅",
            kind=SpanKind.CLIENT
        ) as current_span:
            trace_id = format_trace_id(current_span.get_span_context().trace_id)
            print(f"🔍 Trace ID: {trace_id}")
            print("   -> Azure Portal의 Application Insights에서 확인 가능\n")
            
            # Agent 생성
            async with client.create_agent(
                name="WeatherAgent",
                instructions="당신은 날씨를 알려주는 도움이 되는 조수입니다.",
                tools=get_weather,
            ) as agent:
                question = "서울과 도쿄의 날씨를 비교해줘."
                print(f"[사용자] {question}")
                print(f"[Agent] ", end="")
                
                result = await agent.run(question)
                print(result.text if result.text else "No response")
                
                print("\n✓ Azure AI Agent 실행 완료")
                print("\n📊 Azure Portal에서 확인할 수 있는 정보:")
                print("   - End-to-end 트랜잭션 맵")
                print("   - AI 모델 호출 상세 정보")
                print("   - 토큰 사용량 집계")
                print("   - 성능 병목 지점")
                print("   - 에러 및 예외 추적")

# 실행
await test_azure_ai_observability()

## 6. Custom Span 생성

비즈니스 로직에 맞는 custom span을 생성하여 더 세밀한 추적이 가능합니다.

### 6.1 함수에 Span 추가

In [ ]:
async def process_user_request(user_id: str, query: str) -> str:
    """사용자 요청을 처리하는 비즈니스 로직 (custom span 포함)"""
    
    # Custom span 생성
    with get_tracer().start_as_current_span(
        "process_user_request",
        kind=SpanKind.INTERNAL
    ) as span:
        # Span에 커스텀 속성 추가
        span.set_attribute("user.id", user_id)
        span.set_attribute("query.length", len(query))
        
        print(f"[처리 시작] 사용자 {user_id}의 요청 처리 중...")
        
        # 1단계: 입력 검증
        with get_tracer().start_as_current_span("validate_input"):
            if not query.strip():
                span.set_attribute("validation.result", "failed")
                raise ValueError("빈 쿼리는 처리할 수 없습니다.")
            span.set_attribute("validation.result", "success")
            print("  ✓ 입력 검증 완료")
        
        # 2단계: Agent 처리
        with get_tracer().start_as_current_span("agent_processing"):
            agent = ChatAgent(
                chat_client=OpenAIChatClient(),
                tools=get_weather,
                name="AssistantAgent",
                instructions="사용자의 질문에 도움이 되는 답변을 제공하세요.",
            )
            
            result = await agent.run(query)
            response = result.text if result.text else "응답 없음"
            print("  ✓ Agent 처리 완료")
        
        # 3단계: 결과 후처리
        with get_tracer().start_as_current_span("post_processing"):
            # 응답 길이 제한 (예시)
            if len(response) > 500:
                response = response[:500] + "..."
                span.set_attribute("response.truncated", True)
            
            span.set_attribute("response.length", len(response))
            print("  ✓ 후처리 완료")
        
        print("[처리 완료]\n")
        return response


print("✓ Custom span 함수 정의 완료")

### 6.2 Custom Span 테스트

In [ ]:
async def test_custom_spans():
    """Custom span을 사용한 세밀한 추적"""
    print("=== Custom Span 예제 ===")
    
    with get_tracer().start_as_current_span(
        "시나리오: 사용자 요청 처리 파이프라인",
        kind=SpanKind.CLIENT
    ) as current_span:
        trace_id = format_trace_id(current_span.get_span_context().trace_id)
        print(f"\n🔍 Trace ID: {trace_id}\n")
        
        # 여러 사용자 요청 처리
        requests = [
            ("user_001", "부산의 날씨가 어때?"),
            ("user_002", "오늘 날씨 좋아?"),
        ]
        
        for user_id, query in requests:
            print(f"{'='*60}")
            print(f"사용자: {user_id}")
            print(f"쿼리: {query}")
            print(f"{'='*60}")
            
            response = await process_user_request(user_id, query)
            print(f"[응답] {response}\n")
        
        print("✓ 모든 요청 처리 완료")
        print("\n📊 Custom Span으로 추적된 정보:")
        print("   - 전체 파이프라인 (process_user_request)")
        print("   - 입력 검증 단계 (validate_input)")
        print("   - Agent 처리 단계 (agent_processing)")
        print("   - 후처리 단계 (post_processing)")
        print("   - 각 단계별 속성: user.id, query.length, response.length 등")

# 실행
await test_custom_spans()

## 7. Workflow Observability

Workflow를 사용할 때도 자동으로 telemetry가 수집됩니다.

### 7.1 간단한 Workflow 정의

In [ ]:
from agent_framework import (
    Executor,
    WorkflowBuilder,
    WorkflowContext,
    handler,
)
from typing_extensions import Never


class UpperCaseExecutor(Executor):
    """텍스트를 대문자로 변환하는 Executor"""

    @handler
    async def to_upper_case(self, text: str, ctx: WorkflowContext[str]) -> None:
        print(f"[UpperCaseExecutor] 입력: '{text}'")
        result = text.upper()
        print(f"[UpperCaseExecutor] 출력: '{result}'")
        
        # 다음 executor로 전송
        await ctx.send_message(result)


class ReverseTextExecutor(Executor):
    """텍스트를 뒤집는 Executor"""

    @handler
    async def reverse_text(self, text: str, ctx: WorkflowContext[Never, str]) -> None:
        print(f"[ReverseTextExecutor] 입력: '{text}'")
        result = text[::-1]
        print(f"[ReverseTextExecutor] 출력: '{result}'")
        
        # 최종 결과 출력
        await ctx.yield_output(result)


print("✓ Workflow Executor 정의 완료")

### 7.2 Workflow Observability 테스트

In [ ]:
async def test_workflow_observability():
    """Workflow의 자동 telemetry 수집"""
    print("=== Workflow Observability 예제 ===")
    
    with get_tracer().start_as_current_span(
        "시나리오: Sequential Workflow",
        kind=SpanKind.CLIENT
    ) as current_span:
        trace_id = format_trace_id(current_span.get_span_context().trace_id)
        print(f"\n🔍 Trace ID: {trace_id}\n")
        
        # Executor 생성
        upper_case_executor = UpperCaseExecutor(id="upper_case_executor")
        reverse_text_executor = ReverseTextExecutor(id="reverse_text_executor")
        
        # Workflow 구성
        workflow = (
            WorkflowBuilder()
            .add_edge(upper_case_executor, reverse_text_executor)
            .set_start_executor(upper_case_executor)
            .build()
        )
        
        # Workflow 실행
        input_text = "hello world"
        print(f"입력 텍스트: '{input_text}'\n")
        
        async for event in workflow.run_stream(input_text):
            if hasattr(event, 'output'):
                print(f"\n최종 결과: '{event.output}'")
        
        print("\n✓ Workflow 실행 완료")
        print("\n📊 Workflow Telemetry:")
        print("   - Workflow 빌드 span")
        print("   - Workflow 실행 span")
        print("   - 각 Executor 처리 span")
        print("   - Executor 간 메시지 전송 추적")
        print("   - End-to-end 가시성 제공")

# 실행
await test_workflow_observability()

## 8. Application Insights 연동

Azure Application Insights를 사용하여 프로덕션 환경에서 AI 애플리케이션을 모니터링할 수 있습니다.

### 8.1 설정 방법

#### 방법 1: 환경 변수 사용

`.env` 파일에 다음을 추가:

```bash
ENABLE_OTEL=true
APPLICATIONINSIGHTS_CONNECTION_STRING=InstrumentationKey=...;IngestionEndpoint=...
```

그 다음 `setup_observability()`만 호출하면 자동으로 Application Insights로 전송됩니다.

#### 방법 2: Azure AI Agent Client 사용

```python
async with AzureAIAgentClient(async_credential=credential) as client:
    # Azure AI Project에 연결된 Application Insights 자동 설정
    await client.setup_azure_ai_observability()
```

### 8.2 Application Insights에서 확인할 수 있는 정보

1. **Application Map** 🗺️
   - 전체 시스템 아키텍처 시각화
   - 서비스 간 의존성 파악
   - 병목 지점 식별

2. **Performance** ⚡
   - 요청별 응답 시간
   - AI 모델 호출 지연 시간
   - Tool 실행 성능

3. **Failures** ❌
   - 에러 발생 빈도
   - 예외 스택 추적
   - 실패율 트렌드

4. **Logs** 📝
   - 구조화된 로그 쿼리 (Kusto)
   - 컨텍스트 정보 포함
   - Trace ID 기반 필터링

5. **Custom Metrics** 📊
   - 토큰 사용량 추적
   - 비용 모니터링
   - 사용자별 통계

### 8.3 Kusto 쿼리 예시

```kusto
// 특정 Trace ID로 전체 요청 추적
traces
| where operation_Id == "your-trace-id"
| order by timestamp asc

// AI 모델 호출 성능 분석
dependencies
| where type == "AI"
| summarize avg(duration), count() by name

// 토큰 사용량 집계
customMetrics
| where name == "gen_ai.client.token.usage"
| summarize total_tokens=sum(value) by bin(timestamp, 1h)

// 에러율 트렌드
requests
| summarize 
    total=count(), 
    failures=countif(success == false),
    error_rate=100.0 * countif(success == false) / count()
  by bin(timestamp, 1h)
```

## 9. Aspire Dashboard 사용하기

로컬 개발 시 Aspire Dashboard를 사용하면 Application Insights 없이도 telemetry를 시각화할 수 있습니다.

### 9.1 Aspire Dashboard 시작

터미널에서 실행:

```bash
# Docker를 사용하여 Aspire Dashboard 시작
docker run --rm -it \
  -p 18888:18888 \
  -p 4317:18889 \
  --name aspire-dashboard \
  mcr.microsoft.com/dotnet/aspire-dashboard:9.0
```

브라우저에서 `http://localhost:18888` 접속

### 9.2 환경 변수 설정

`.env` 파일:

```bash
ENABLE_OTEL=true
ENABLE_SENSITIVE_DATA=true  # 개발 환경에서만!
OTLP_ENDPOINT=http://localhost:4317
```

### 9.3 Aspire Dashboard 특징

✅ **Traces 탭**: 전체 요청 추적, Span 계층 구조  
✅ **Metrics 탭**: 실시간 메트릭 차트  
✅ **Logs 탭**: 구조화된 로그 검색  
✅ **Structured 탭**: 속성 기반 필터링  
✅ **No Azure Required**: 완전 로컬 환경  

## 요약 및 베스트 프랙티스

### Observability 핵심 개념

1. **OpenTelemetry 3대 요소**
   - Traces: 요청 경로 추적
   - Logs: 이벤트 기록
   - Metrics: 수치 데이터

2. **자동 계측**
   - Agent Framework가 자동으로 telemetry 생성
   - GenAI Semantic Conventions 준수
   - 토큰 사용량, 응답 시간 자동 추적

3. **설정 방법**
   - `setup_observability()`: 환경 변수 기반 자동 설정
   - `setup_azure_ai_observability()`: Azure AI 자동 연동
   - Manual setup: 커스텀 exporter 설정

### 베스트 프랙티스

#### 1. 환경별 설정

```python
# ✓ Good: 개발 환경
# .env
ENABLE_OTEL=true
ENABLE_SENSITIVE_DATA=true  # 개발에서만!
OTLP_ENDPOINT=http://localhost:4317  # Aspire Dashboard

# ✓ Good: 프로덕션 환경
# .env
ENABLE_OTEL=true
ENABLE_SENSITIVE_DATA=false  # 프로덕션에서는 false!
APPLICATIONINSIGHTS_CONNECTION_STRING=InstrumentationKey=...
```

#### 2. Custom Span 사용

```python
# ✓ Good: 비즈니스 로직에 의미 있는 span 추가
with get_tracer().start_as_current_span(
    "process_payment",
    kind=SpanKind.INTERNAL
) as span:
    span.set_attribute("payment.amount", amount)
    span.set_attribute("payment.currency", "USD")
    # 비즈니스 로직

# ✗ Bad: 너무 세밀한 span은 오버헤드 증가
with get_tracer().start_as_current_span("add_two_numbers"):
    result = a + b  # 불필요함
```

#### 3. Trace ID 활용

```python
# ✓ Good: 사용자에게 Trace ID 제공
with get_tracer().start_as_current_span("user_request") as span:
    trace_id = format_trace_id(span.get_span_context().trace_id)
    print(f"요청 ID: {trace_id}")
    # 사용자가 이 ID로 지원팀에 문의 가능
```

#### 4. 민감한 데이터 관리

```python
# ✓ Good: 프로덕션에서는 민감한 데이터 비활성화
if os.getenv("ENVIRONMENT") == "production":
    # ENABLE_SENSITIVE_DATA=false
    # 프롬프트와 응답 내용이 로깅되지 않음
    pass

# ✗ Bad: 프로덕션에서 민감한 데이터 활성화
# ENABLE_SENSITIVE_DATA=true  # 절대 안 됨!
```

#### 5. 로그 레벨 설정

```python
# ✓ Good: 환경별 로그 레벨
import logging

if os.getenv("ENVIRONMENT") == "development":
    logging.getLogger().setLevel(logging.DEBUG)
else:
    logging.getLogger().setLevel(logging.INFO)
```

### 모니터링 전략

| 환경 | Observability 도구 | 설정 |
|------|------------------|------|
| 로컬 개발 | Aspire Dashboard | OTLP_ENDPOINT |
| 개발/스테이징 | Application Insights | APPLICATIONINSIGHTS_CONNECTION_STRING |
| 프로덕션 | Application Insights + 알림 | 모든 telemetry + 알림 규칙 |
| CI/CD | Console Output | ENABLE_OTEL=true (exporter 없음) |

### 알림 설정 (Application Insights)

프로덕션에서 설정할 알림:

1. **에러율**: 5분간 에러율 > 5%
2. **응답 시간**: P95 > 3초
3. **토큰 사용량**: 시간당 토큰 > 임계값
4. **가용성**: 가동 시간 < 99%

### 다음 단계

1. **Aspire Dashboard 설정**: 로컬 개발 환경 구축
2. **Custom Attributes**: 비즈니스 메트릭 추가
3. **Application Insights**: Azure 환경 연동
4. **알림 규칙**: 프로덕션 모니터링 설정
5. **대시보드 구축**: Kusto 쿼리로 커스텀 대시보드
6. **비용 최적화**: 토큰 사용량 분석 및 최적화

### 참고 자료

- [Azure Monitor OpenTelemetry](https://learn.microsoft.com/azure/azure-monitor/app/opentelemetry-overview)
- [Aspire Dashboard](https://learn.microsoft.com/dotnet/aspire/fundamentals/dashboard/overview)
- [OpenTelemetry Python](https://opentelemetry.io/docs/languages/python/)
- [GenAI Semantic Conventions](https://opentelemetry.io/docs/specs/semconv/gen-ai/)